<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_experimental_heft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [1]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [2]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [3]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 83.8 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20443 sha256=d65d4fa86ab8a5ea8b2ec6a2f2bfa6c80ddfe1c8b9af3c145e86f88551d14637
  Stored in directory: /root/.cache/pip/wheels/6a/65/c4/d27eeee9ba3fc150a0dae150519591103b9e0dbffde3ae77dc
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3530 sha256=2f0e03ade6c66ae72ca4350b89efec0ab650023e431d16594caab5d9ed5b2693
  Stored in directory: /root/.cache/pip/wheels/ba/d9/a0/ddd93af16d5855dd9bad417623e70948fdac119d1d34fb17c8
Successfully built fst-pso miniful
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


### Downloads

In [3]:
data_version = 'football_live_221229/'
project = neptune.init_project(
    name="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'data.rar'].download('./data.rar')
project[data_version + 'info.rar'].download('./info.rar')
project[data_version + 'prem.rar'].download('./prem.rar')
project[data_version + 'train_id.csv'].download('./train_id.csv')
project[data_version + 'test_id.csv'].download('./test_id.csv')
project[data_version + 'holdout_id.csv'].download('./holdout_id.csv')
project[data_version + 'data_train.csv.gz'].download('./data_train.csv.gz')
project[data_version + 'data_test.csv.gz'].download('./data_test.csv.gz')
project[data_version + 'data_hold.csv.gz'].download('./data_hold.csv.gz')

#del project[data_version]
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


### Imports

In [3]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

#import dask.dataframe as dd
import subprocess
import sys
from glob import glob
from tqdm import tqdm

1.3.5
1.21.6


In [4]:
import plotly.express as px

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

### Code

#####Functions

In [6]:
def run_bash(bashCommand:str, nameCommand = ''):
        process = subprocess.Popen([bashCommand], 
                           shell=True)
        _, error = process.communicate()
        if error:
            print(f'{nameCommand} error:\n', error)

#### Unpack

In [8]:
for file in glob('./*'):    
    if '.rar' in file:
        print(file)
        bash = f'unrar e {file} && rm {file}'
        run_bash(bash)

./data.rar
./prem.rar
./info.rar


### _

In [51]:
data_df = pd.read_csv(
    './data_train.csv.gz', 
    dtype = {
        'Id':np.int32, 'Minute':np.int16, 'Active':np.int8,
        'Score1':np.int16, 'Score2':np.int16, 'A1':np.int16, 'A2':np.int16,
        'DA1':np.int16, 'DA2':np.int16,'Pos1':np.float32, 'Pos2':np.float32,
        'Off1':np.int16, 'Off2':np.int16,'On1':np.int16, 'On2':np.int16,
        'YC1':np.float32, 'YC2':np.float32, 'RC1':np.float32, 'RC2':np.float32,
        'Sub1':np.float32, 'Sub2':np.float32, 'Pen1':np.float32, 'Pen2':np.float32,
        'Cor1':np.float32, 'Cor2':np.float32,
        }
    )

In [53]:
data_df = data_df[data_df['Id'] != 1898989]

In [14]:
# Присоединяем итоговый результат и по первому тайму
def add_match_results (data_df:pd.DataFrame, cols = ['Id', 'Result1', 'Result2', 'Periods'], info_path = './info.csv'):
    info_df = pd.read_csv(info_path, sep = ';', usecols = cols)
    info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)
    info_df[['Time1Res1', 'Time1Res2']] = info_df['Period1'].str.split(':', expand = True)
    info_df = info_df[~info_df['Id'].duplicated(keep = False)]
    period1_result_dict = info_df.set_index('Id')[['Time1Res1', 'Time1Res2', 'Result1', 'Result2']].to_dict(orient = 'index')
    #data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] =  \
    return [[period1_result_dict[id]['Time1Res1'], period1_result_dict[id]['Time1Res1'], 
    period1_result_dict[id]['Result1'], period1_result_dict[id]['Result2']] 
    if id in period1_result_dict else [None, None, None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]
    

In [55]:
data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] = add_match_results (data_df)
isdigit = [str(a) for a in range(100)]
data_df[~data_df['Time1Res1'].isin(isdigit)] = -1
data_df[~data_df['Time1Res2'].isin(isdigit)] = -1
data_df[['Time1Res1', 'Time1Res2']] = data_df[['Time1Res1', 'Time1Res2']].astype(np.int16)
data_df[['Result1', 'Result2']] = data_df[['Result1', 'Result2']].astype(np.int16)
data_df = data_df.drop(['Time1Res1', 'Time1Res2'], axis = 1)

100%|██████████| 10391998/10391998 [00:25<00:00, 402261.70it/s]


In [38]:
# Добавляем прематчевые линии
def add_match_lines (data_df:pd.DataFrame, cols = ['P1', 'PX', 'P2', 'PR'], prem_path = './prem.csv'):
    prem_df = pd.read_csv(prem_path, sep = ';')
    prem_df = prem_df[~prem_df['Id'].duplicated(keep = False)]
    prem_dict = prem_df.set_index('Id')[cols].to_dict(orient = 'index')
    #data_df.loc[:,['P1', 'PX', 'P2', 'PR']] =  \
    return [[prem_dict[id]['P1'], prem_dict[id]['PX'], prem_dict[id]['P2'], prem_dict[id]['PR']] 
    if id in prem_dict else [None, None, None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]

In [57]:
data_df.loc[:,['P1', 'PX', 'P2', 'PR']] = add_match_lines (data_df)
data_df[['P1', 'PX', 'P2', 'PR']] = data_df[['P1', 'PX', 'P2', 'PR']].astype(np.float32)
data_df = data_df.drop(['PX', 'PR'], axis = 1)

100%|██████████| 10391998/10391998 [00:23<00:00, 447923.26it/s]


In [52]:
def transform_dataset(data_df:pd.DataFrame, remain = []]):
    input_cols_set = set(data_df.columns)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'] / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill')
    data_df['Score1_norm'] = [f / 4 if f < 4 else 1.0 for f in data_df['Score1_norm'].values]
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill')
    data_df['Score2_norm'] = [f / 4 if f < 4 else 1.0 for f in data_df['Score2_norm'].values]

    data_df['Score_diff'] = data_df['Score1'] - data_df['Score2']
    data_df['Score_diff'] = [-4 if d < -4 else 4 if d > 4 else d for d in data_df['Score_diff']]
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = [d / 4 for d in data_df['Score_diff'].values]
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['A1'].values]
    data_df['A2_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['A2'].values]
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'] / data_df['Minute']
    data_df['A1perMIN'] = [a1 if a1 < 4 else 4 for a1 in data_df['A1perMIN'].values]
    data_df['A2perMIN'] = data_df['A2'] / data_df['Minute']
    data_df['A2perMIN'] = [a2 if a2 < 4 else 4 for a2 in data_df['A2perMIN'].values]
    # динамика атак
    data_df['A1relativ'] = data_df['A1'] - data_df['A1'].shift(5)
    data_df.loc[data_df['A1relativ'] < 0, ['A1relativ']] = 0
    data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
    data_df['A1relativ'] = [a if a < 15 else 15 for a in data_df['A1relativ'].values]
    data_df['A2relativ'] = data_df['A2'] - data_df['A2'].shift(5)
    data_df.loc[data_df['A2relativ'] < 0, ['A2relativ']] = 0
    data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
    data_df['A2relativ'] = [a if a < 15 else 15 for a in data_df['A2relativ'].values]
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['DA1'].values]
    data_df['DA2_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['DA2'].values]
    # опасные атаки в минуту    
    data_df['DA1perMIN'] = data_df['DA1'] / data_df['Minute']
    data_df['DA1perMIN'] = [da1 if da1 < 3 else 3 for da1 in data_df['DA1perMIN'].values]
    data_df['DA2perMIN'] = data_df['DA2'] / data_df['Minute']
    data_df['DA2perMIN'] = [da2 if da2 < 3 else 3 for da2 in data_df['DA2perMIN'].values]
    # динамика опасных атак
    data_df['DA1relativ'] = data_df['DA1'] - data_df['DA1'].shift(5)
    data_df.loc[data_df['DA1relativ'] < 0, ['DA1relativ']] = 0
    data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
    data_df['DA1relativ'] = [a if a < 10 else 10 for a in data_df['DA1relativ'].values]
    data_df['DA2relativ'] = data_df['DA2'] - data_df['DA2'].shift(5)
    data_df.loc[data_df['DA2relativ'] < 0, ['DA2relativ']] = 0
    data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
    data_df['DA2relativ'] = [a if a < 10 else 10 for a in data_df['DA2relativ'].values]
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill')
    data_df['Pos1_cleaned'] = [0.2 if p < 20 else 0.8 if p > 80 else p / 100 for p in data_df['Pos1_cleaned'].values]
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill')
    data_df['Pos2_cleaned'] = [0.2 if p < 20 else 0.8 if p > 80 else p / 100 for p in data_df['Pos2_cleaned'].values]
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill')
    data_df['Off1_norm'] = [f / 10 if f < 10 else 1.0 for f in data_df['Off1_norm'].values]
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill')
    data_df['Off2_norm'] = [f / 10 if f < 10 else 1.0 for f in data_df['Off2_norm'].values]
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill')
    data_df['On1_norm'] = [f / 5 if f < 5 else 1.0 for f in data_df['On1_norm'].values]
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill')
    data_df['On2_norm'] = [f / 5 if f < 5 else 1.0 for f in data_df['On2_norm'].values]
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df['YC1_transformed'] = [y / 2 if y < 2 else 1.0 for y in data_df['YC1'].values]
    data_df['YC2_transformed'] = [y / 2 if y < 2 else 1.0 for y in data_df['YC2'].values]
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df['RC1_transformed'] = [y if y < 1 else 1 for y in data_df['RC1'].values]
    data_df['RC2_transformed'] = [y if y < 1 else 1 for y in data_df['RC2'].values]
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df['Sub1_transformed'] = [y if y < 1 else 1 for y in data_df['Sub1'].values]
    data_df['Sub2_transformed'] = [y if y < 1 else 1 for y in data_df['Sub2'].values]
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df['Cor1_transformed'] = [y / 6 if y < 6 else 1.0 for y in data_df['Cor1'].values]
    data_df['Cor2_transformed'] = [y / 6 if y < 6 else 1.0 for y in data_df['Cor2'].values]
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'])
    data_df['P2_transformed'] = np.log(data_df['P2']) / 2
    print('12. Линии посчитаны...')
    if remain != 'all':
        output_cols_list = remain + list(set(data_df.columns) - input_cols_set)
        return data_df[output_cols_list]

    return data_df

In [14]:
data_df = transform_dataset(data_df) #, remain = ['Id', 'Time1Res1', 'Time1Res2', 'Result1', 'Result2'])

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...


In [14]:
data_df['P1_transformed'] = np.log(data_df['P1'])
data_df['P2_transformed'] = np.log(data_df['P2']) / 2

/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [80]:
drop_transformed = True

In [59]:
data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50

In [82]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10391998 entries, 0 to 10392039
Data columns (total 40 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Id           int64  
 1   Minute       int16  
 2   Active       int8   
 3   A1           int16  
 4   A2           int16  
 5   DA1          int16  
 6   DA2          int16  
 7   Pos1         float32
 8   Pos2         float32
 9   Off1         int16  
 10  Off2         int16  
 11  On1          int16  
 12  On2          int16  
 13  YC1          float32
 14  YC2          float32
 15  RC1          float32
 16  RC2          float32
 17  Sub1         float32
 18  Sub2         float32
 19  Pen1         float32
 20  Pen2         float32
 21  Cor1         float32
 22  Cor2         float32
 23  Result1      int16  
 24  Result2      int16  
 25  P1           float32
 26  P2           float32
 27  min_norm     float32
 28  Score1_norm  float32
 29  Score2_norm  float32
 30  Score_diff   float32
 31  Score_cat_1  uint8  
 

In [68]:
data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

In [77]:
data_df['Score_diff'] = data_df['Score1'].astype(np.float32) - data_df['Score2'].astype(np.float32)
data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)

In [81]:
if drop_transformed:
    data_df = data_df.drop(['Score1', 'Score2'], axis = 1)

In [18]:
data_df['A1_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['A1'].values]
data_df['A2_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['A2'].values]

In [19]:
data_df['A1perMIN'] = data_df['A1'] / data_df['Minute']
data_df['A1perMIN'] = [a1 if a1 < 4 else 4 for a1 in data_df['A1perMIN'].values]
data_df['A2perMIN'] = data_df['A2'] / data_df['Minute']
data_df['A2perMIN'] = [a2 if a2 < 4 else 4 for a2 in data_df['A2perMIN'].values]

In [20]:
data_df['A1relativ'] = data_df['A1'] - data_df['A1'].shift(5)
data_df.loc[data_df['A1relativ'] < 0, ['A1relativ']] = 0
data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
data_df['A1relativ'] = [a if a < 15 else 15 for a in data_df['A1relativ'].values]
data_df['A2relativ'] = data_df['A2'] - data_df['A2'].shift(5)
data_df.loc[data_df['A2relativ'] < 0, ['A2relativ']] = 0
data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
data_df['A2relativ'] = [a if a < 15 else 15 for a in data_df['A2relativ'].values]

In [21]:
if drop_transformed:
    data_df = data_df.drop(['A1', 'A2'], axis = 1)

In [22]:
data_df['DA1_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['DA1'].values]
data_df['DA2_scaled'] = [x /100 if x < 80 else (80 + (x - 80) / 5) / 100 for x in data_df['DA2'].values]

In [23]:
data_df['DA1perMIN'] = data_df['DA1'] / data_df['Minute']
data_df['DA1perMIN'] = [da1 if da1 < 3 else 3 for da1 in data_df['DA1perMIN'].values]
data_df['DA2perMIN'] = data_df['DA2'] / data_df['Minute']
data_df['DA2perMIN'] = [da2 if da2 < 3 else 3 for da2 in data_df['DA2perMIN'].values]

In [24]:
data_df['DA1relativ'] = data_df['DA1'] - data_df['DA1'].shift(5)
data_df.loc[data_df['DA1relativ'] < 0, ['DA1relativ']] = 0
data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
data_df['DA1relativ'] = [a if a < 10 else 10 for a in data_df['DA1relativ'].values]
data_df['DA2relativ'] = data_df['DA2'] - data_df['DA2'].shift(5)
data_df.loc[data_df['DA2relativ'] < 0, ['DA2relativ']] = 0
data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
data_df['DA2relativ'] = [a if a < 10 else 10 for a in data_df['DA2relativ'].values]

In [25]:
if drop_transformed:
    data_df = data_df.drop(['DA1', 'DA2'], axis = 1)

In [26]:
data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill')
data_df['Pos1_cleaned'] = [0.2 if p < 20 else 0.8 if p > 80 else p / 100 for p in data_df['Pos1_cleaned'].values]
data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill')
data_df['Pos2_cleaned'] = [0.2 if p < 20 else 0.8 if p > 80 else p / 100 for p in data_df['Pos2_cleaned'].values]

In [27]:
if drop_transformed:
    data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)

In [28]:
data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill')
data_df['Off1_norm'] = [f / 10 if f < 10 else 1.0 for f in data_df['Off1_norm'].values]
data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill')
data_df['Off2_norm'] = [f / 10 if f < 10 else 1.0 for f in data_df['Off2_norm'].values]

In [29]:
if drop_transformed:
    data_df = data_df.drop(['Off1', 'Off2'], axis = 1)

In [30]:
data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill')
data_df['On1_norm'] = [f / 5 if f < 5 else 1.0 for f in data_df['On1_norm'].values]
data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill')
data_df['On2_norm'] = [f / 5 if f < 5 else 1.0 for f in data_df['On2_norm'].values]

In [31]:
if drop_transformed:
    data_df = data_df.drop(['On1', 'On2'], axis = 1)

In [32]:
data_df['YC1_transformed'] = [y / 2 if y < 2 else 1.0 for y in data_df['YC1'].values]
data_df['YC2_transformed'] = [y / 2 if y < 2 else 1.0 for y in data_df['YC2'].values]

In [ ]:
if drop_transformed:
    data_df = data_df.drop(['YC1', 'YC2'], axis = 1)

In [33]:
data_df['RC1_transformed'] = [y if y < 1 else 1 for y in data_df['RC1'].values]
data_df['RC2_transformed'] = [y if y < 1 else 1 for y in data_df['RC2'].values]

In [34]:
if drop_transformed:
    data_df = data_df.drop(['RC1', 'RC2'], axis = 1)

In [35]:
data_df['Sub1_transformed'] = [y if y < 1 else 1 for y in data_df['Sub1'].values]
data_df['Sub2_transformed'] = [y if y < 1 else 1 for y in data_df['Sub2'].values]

In [36]:
if drop_transformed:
    data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)

In [37]:
data_df['Cor1_transformed'] = [y / 6 if y < 6 else 1.0 for y in data_df['Cor1'].values]
data_df['Cor2_transformed'] = [y / 6 if y < 6 else 1.0 for y in data_df['Cor2'].values]

In [38]:
if drop_transformed:
    data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)

In [39]:
data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32)
data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2

In [40]:
if drop_transformed:
    data_df = data_df.drop(['P1', 'PX', 'P2', 'PR'], axis = 1)

In [41]:
data_df.to_csv('./train_transformed.csv.gz', index = False, compression = {'method':'gzip'})

In [34]:
data_df.to_csv('data_train_transformed.csv.gz', index = False, compression={'method': 'gzip'})

In [ ]:
px.histogram(np.log(data_df['P2'].iloc[:1_000_000]) / 2)

In [ ]:
px.histogram(np.log(data_df['P1'].iloc[:1_000_000]))

In [7]:
data_df = pd.read_csv('./data.csv', sep = ';',
                      #nrows = 10_000_000, 
                      usecols = ['Id', 'Minute', 'Score1', 'Score2', 
                                 'A1', 'A2', 'DA1', 'DA2', 'Pos1', 'Pos2', 'Off1', 
                                 'Off2', 'On1', 'On2', 'YC1', 'YC2', 'RC1', 'RC2', 
                                 'Sub1', 'Sub2', 'Pen1', 'Pen2', 'Cor1', 'Cor2'])

In [ ]:
data_df.head(100)

In [39]:
!head /content/info.csv

Id;BeginTime;Home;Away;League;Country;Type;Result1;Result2;Periods;HomeId;AwayId;LeagueId;Season;Round
358904;01.01.2018 22:45:00;Bromley;Ebbsfleet United;England National League;gb;;4;2;3:1,4:2;;;;;
330509;01.01.2018 22:00:00;Hapoel Bnei Sakhnin;Maccabi Tel Aviv;Israel Premier League;il;;0;1;0:0,0:1;;;;;
575487;01.01.2018 20:30:00;Aston Villa;Bristol City;England Championship;gb;;5;0;2:0,5:0;;;;;
338563;01.01.2018 20:30:00;Everton;Man Utd;England Premier League;gb;;0;2;0:0,0:2;;;;;
340310;01.01.2018 20:15:00;Bangor City;Llandudno;Wales Premier League;gb;;0;1;0:0,0:1;;;;;
577973;01.01.2018 20:13:49;JKU FC;Taifa Jang’ombe;World Club Friendlies;;;0;0;0:0,0:0;;;;;
577112;01.01.2018 20:00:00;Hapoel Kfar Saba;Hapoel Ramat Gan;Israel Leumit Liga;il;;0;0;0:0,0:0;;;;;
571744;01.01.2018 20:00:00;Maccabi Achi Nazareth;Beitar Tel Aviv Bat Yam;Israel Leumit Liga;il;;0;0;0:0,0:0;;;;;
571518;01.01.2018 20:00:00;Ironi Ramat Hasharon;Hapoel Hadera;Israel Leumit Liga;il;;0;4;0:2,0:4;;;;;


In [9]:
info_df = pd.read_csv('./info.csv', sep = ';', usecols = ['Id', 'Result1', 'Result2', 'Periods'])

In [15]:
info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)

In [29]:
miss_p1_list = list(info_df[info_df['Period1'] == '']['Id'])

In [35]:
period1_result_dict = info_df.set_index('Id')['Period1'].to_dict()

In [36]:
data_df.loc[:, 'periods1'] =  [period1_result_dict[id] if id in period1_result_dict else '' for id in data_df['Id'].values]


In [42]:
(data_df['periods1'] == '').sum()

4247

In [40]:
!head /content/prem.csv

Id;IsPrem;P1;PX;P2;PR;PRTime;IsKO;KO1;KOX;KO2;KOR;KOTime;KOminute;KOscore
358904;1;2.6;3.2;2.8;0.0543;01.01.2018 22:20:17;0;;;;;;;
330509;1;9.5;4.5;1.364;0.0606;01.01.2018 21:58:29;0;;;;;;;
575487;1;2.4;3.3;3.3;0.0227;01.01.2018 20:09:47;0;;;;;;;
338563;1;5.75;3.75;1.7;0.0288;01.01.2018 19:51:18;0;;;;;;;
340310;1;1.571;4;5.75;0.0605;01.01.2018 20:03:11;0;;;;;;;
577973;1;2;3.25;3.3;0.1107;31.12.2017 20:10:35;0;;;;;;;
577112;1;1.8;3.4;4.333;0.0805;01.01.2018 18:14:51;0;;;;;;;
571744;1;2.45;3.2;2.7;0.091;01.01.2018 12:36:33;0;;;;;;;
571518;1;1.909;3.3;3.75;0.0935;01.01.2018 19:37:02;0;;;;;;;


In [10]:
prem_df = pd.read_csv('./prem.csv', sep = ';')

In [45]:
prem_df

,Id,IsPrem,P1,PX,P2,PR,PRTime,IsKO,KO1,KOX,KO2,KOR,KOTime,KOminute,KOscore
0,358904,1,2.600,3.200,2.800,0.0543,01.01.2018 22:20:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,330509,1,9.500,4.500,1.364,0.0606,01.01.2018 21:58:29,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,575487,1,2.400,3.300,3.300,0.0227,01.01.2018 20:09:47,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,338563,1,5.750,3.750,1.700,0.0288,01.01.2018 19:51:18,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,340310,1,1.571,4.000,5.750,0.0605,01.01.2018 20:03:11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749195,6058161,1,2.750,4.000,2.000,0.1136,26.12.2022 9:54:23,1.0,2.875,4.0,2.0,0.0978,26.12.2022 10:01:18,1.0,0.0
749196,5404664,1,2.800,3.600,2.450,0.0431,26.12.2022 9:59:53,1.0,2.750,3.6,2.5,0.0414,26.12.2022 10:01:30,1.0,0.0
749197,6063457,1,1.166,6.000,11.000,0.1152,26.12.2022 7:39:37,1.0,1.181,6.0,10.0,0.1134,26.12.2022 7:42:01,1.0,0.0
749198,6063456,1,1.083,8.500,23.000,0.0845,26.12.2022 0:36:08,1.0,1.090,8.5,21.0,0.0827,26.12.2022 5:01:48,1.0,0.0


In [ ]:
prem_df[prem_df['IsPrem'] != 1]